# データ解析演習　第2回課題
## 学籍番号・氏名
** 必ず記入すること **
```
J2XXYYY
XX YY
```

- 作業の途中で気がついたことがあれば，Markdown block を追加して記入すること．
- 試してみるために Code block を追加しても構わない．

作業後に保存済みのファイルを  
http://nip.info.kogakuin.ac.jp/lectures/2016/data_analysis　で提出すること．

## 基本
- 「+」ボタンで新しい block を作れる
- 上下矢印ボタンで block の順番を入れ替えることができる
- ブロックの種類をプルダウンにある「Markdown」と「Code」で切り替えることができる．


- Markdown　の補足
 - 「空行を入れる」あるいは「行末に空白を2つ入れる」で改行

## CSV ファイルの読み込みと表示
- pandas ライブラリを利用するために import を行う
  - C 言語の include のような物
- csv ファイルの読み込みには pandas ライブラリの read_csv 関数を使う
  - read_csv は DataFrame という形式で読み込んだ結果を返す
- 結果を変数 data に代入する

In [ ]:
import pandas
data = pandas.read_csv('data02.csv')
data.head()

## データの説明

気象庁からのデータを少し加工しました．
- **day**　　年月日
- **average**　　平均気温
- **max**　　最高気温
- **min**　　最低気温
- **cloud**　　平均雲量
- **wind**　　風速

### References  
http://www.data.jma.go.jp/gmd/risk/obsdl/index.php

## 修正
今回は年月日ごとのデータになっているので index と呼ばれる各データの主ラベルを 'day' にして，　　
きちんと日付として認識させる．今回はあまり重要ではないが参考までに．

- **index_col**  どの列を index にするか．  
    指定しない場合は，通し番号をつける．
- **parse_dates**　　日付の解析を指定する．  
    True だと index について解析する．  
    その他の使い方は pandas.read_csv を検索すること

In [ ]:
data = pandas.read_csv('data02-2.csv', index_col='day', parse_dates=True)
data.head()

## グラフの表示
- グラフを表示するために %matplotlib inline を実行する.
- matplotlib のグラフ機能を利用するために matplotlib.pyplot ライブラリを読み込み
  - ライブラリ名が長いので as plt で matplotlib.pyplot.XXX の代わりに plt.XXX と記述できるようにする．


- 前回と同様の plot を利用して折れ線グラフ（kind 指定なし）と散布図（scatter）を表示する


- In を実行した最後の行の内容が Out に表示される．
- plot を複数回行うと，内部に保存され，Out の後にまとめて表示される．

- plt.sho() を実行するとその場で表示される．（flush）


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
data.plot(y=['average', 'min', 'max'])
plt.show()
data.plot(kind='scatter', x='min', y='max')
plt.show()

### Seaborn の利用
- 統計に特化したグラフライブラリである seaborn を読み込む
  - 一般的に別名を sns とすることが多い．
  - 読み込むだけで pandas の標準的なグラフの表示が変わる

それ以外に，
- 等高線
- x, y へのヒストグラムの追加

など様々な機能がある．


- グラフを変数に保存して set_xlim, set_ylim を呼び出すと x,y の表示範囲を指定できる．
  - そのまま実行した場合と，コメントを外した場合を比較する

In [ ]:
import seaborn as sns
ax = data.plot(kind='scatter', x='min', y='max')
plt.show()
ax = sns.kdeplot(data['min'], data['max'])
#ax.set_xlim(-5,30)
#ax.set_ylim(0,40)
plt.show()

- グラフ ax を使いまわして重ね描きできる．

In [ ]:
ax = data.plot(kind='scatter', x='min', y='max')
seaborn.kdeplot(data['min'], data['max'], ax=ax)
plt.show()

- x, y にヒストグラムを付け加える．

In [ ]:
seaborn.jointplot(data['min'], data['max'])
plt.show()

## 多変量正規分布での近似
- データから統計量を計算
  - 平均 mean
  - 分散／共分散 cov  
- 確率分布を表示するために，x,y,p を用意
- 等高線と散布図を重ねて表示


- 通常のデータ解析ではこのような手順は踏まないのでプログラムが少しややこしくなっている．
- 完全に理解する必要はないが，少しでも理解を深めるために d, mean, cov, x, y, xy, p がどのような内容を確認すること．

In [ ]:
import numpy as np
import scipy.stats as st

# 今回は 'min' と 'max' のみに注目，平均と分散共分散行列を計算
mean = data[['min','max']].mean()
cov = data[['min','max']].cov()

# x 座標と y 座標　を grid で得る
x, y = np.mgrid[-5:30:1, 0:40:1]
# x 座標と y 座標をまとめる
xy = np.stack((x,y),-1)
# 平均 mean, 分散共分散行列 cov の多変量正規分布の座標ごとの尤度 (pdf; probability distribution function)
p = st.multivariate_normal(mean, cov).pdf(xy)

# 尤度の等高線と散布図を重ね描き
plt.contour(x, y, p, cmap='Reds')
plt.scatter(data['min'], data['max'])
plt.show()

In [ ]:
# いろいろな変数の内容を Out に表示させること
x

### コードの解析
#### わかった範囲で説明をすること

## 課題
min, max の組み合わせ以外（例えば, wind と max，cloud と max など）について
- 散布図と等高線の表示
- 平均と分散共分散行列を用いて多変量正規分布で近似した場合の等高線

を描き，多変量正規分布で近似する場合の注意点について考察せよ．


例えば，
- 平均と分散共分散だけを用いて予測を行うとどのような危険があるか．
- 危険性を避けるためにはどのようなことをすれば良いか．
- 正規分布が使えない時はどのような方法が考えられるか．

### コード

### 考察

#### この下に記入


### 感想・質問
#### この下に記入